In [18]:
import psycopg2
import yaml
import pandas as pd

In [19]:
def read_yaml(file_path):
    """Читает YAML файл и возвращает содержимое."""
    with open(file_path, 'r') as file:
        return yaml.safe_load(file)

In [20]:
# Пример использования
config = read_yaml('connection.yaml')

print(config)

{'otus_postgreSQL': {'db': 'demo', 'host': '45.134.254.239', 'port': 25432, 'user': 'Student_01', 'pass': 123456789}}


In [23]:
config['otus_postgreSQL']['host']

'45.134.254.239'

In [24]:
conn = None

try:
    conn = psycopg2.connect(
                            host=config['otus_postgreSQL']['host'],
                            port=config['otus_postgreSQL']['port'],
                            database=config['otus_postgreSQL']['db'],
                            user=config['otus_postgreSQL']['user'],
                            password=str(config['otus_postgreSQL']['pass']))
    cur = conn.cursor()

    # execute a statement
    print('PostgreSQL database version:')
    cur.execute('SELECT version()')

    # display the PostgreSQL database server version
    db_version = cur.fetchone()
    print(db_version)
    
    # close the communication with the PostgreSQL
    cur.close()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()
        print('Database connection closed.')

PostgreSQL database version:
('PostgreSQL 14.2 (Debian 14.2-1.pgdg110+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 10.2.1-6) 10.2.1 20210110, 64-bit',)
Database connection closed.


In [25]:
def query(query_string):
    try:
        conn = psycopg2.connect(
                                host=config['otus_postgreSQL']['host'],
                                port=config['otus_postgreSQL']['port'],
                                database=config['otus_postgreSQL']['db'],
                                user=config['otus_postgreSQL']['user'],
                                password=str(config['otus_postgreSQL']['pass']))
        cur = conn.cursor()

        # execute a statement
        cur.execute(query=query_string)

        # display the PostgreSQL database server version
        results = cur.fetchall()
        
        # close the communication with the PostgreSQL
        # cur.close()
        return results
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        return -1
    finally:
        if conn is not None:
            conn.close()
        

In [26]:
query("select tablename \
        from pg_catalog.pg_tables \
        where schemaname = 'bookings';")

[('ticket_flights',),
 ('airports_data',),
 ('tickets',),
 ('boarding_passes',),
 ('flights',),
 ('aircrafts_data',),
 ('bookings',),
 ('seats',)]

In [27]:
query("select count(*) \
    from flights f ;")

[(65664,)]

Получим данные из таблицы

In [28]:
res = query("select * \
      from flights \
      limit 100;")

In [29]:
res[0]

(182,
 'PG0402',
 datetime.datetime(2017, 9, 1, 9, 25, tzinfo=datetime.timezone.utc),
 datetime.datetime(2017, 9, 1, 10, 20, tzinfo=datetime.timezone.utc),
 'DME',
 'LED',
 'Scheduled',
 '321',
 None,
 None)

Получим заголовки таблицы

In [33]:
columns = query("SELECT column_name \
FROM information_schema.columns \
WHERE table_schema = 'bookings' \
  AND table_name   = 'flights';")

# уберем лишние данные из результата
columns = [x[0] for x in columns]
columns

['flight_id',
 'flight_no',
 'scheduled_departure',
 'scheduled_arrival',
 'departure_airport',
 'arrival_airport',
 'status',
 'aircraft_code',
 'actual_departure',
 'actual_arrival']

In [34]:
flights = pd.DataFrame(res, columns=columns)
flights

,flight_id,flight_no,scheduled_departure,scheduled_arrival,departure_airport,arrival_airport,status,aircraft_code,actual_departure,actual_arrival
0,182,PG0402,2017-09-01 09:25:00+00:00,2017-09-01 10:20:00+00:00,DME,LED,Scheduled,321,NaT,NaT
1,1996,PG0335,2017-08-26 06:30:00+00:00,2017-08-26 08:35:00+00:00,DME,JOK,Scheduled,CN1,NaT,NaT
2,5979,PG0384,2017-08-26 09:10:00+00:00,2017-08-26 09:40:00+00:00,DME,BZK,Scheduled,SU9,NaT,NaT
3,8136,PG0138,2017-08-28 07:15:00+00:00,2017-08-28 08:20:00+00:00,VKO,RTW,Scheduled,CR2,NaT,NaT
4,10455,PG0277,2017-09-12 08:45:00+00:00,2017-09-12 12:10:00+00:00,SVO,OVB,Scheduled,773,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...
95,68,PG0405,2017-05-17 06:35:00+00:00,2017-05-17 07:30:00+00:00,DME,LED,Arrived,321,2017-05-17 06:36:00+00:00,2017-05-17 07:32:00+00:00
96,69,PG0402,2017-05-27 09:25:00+00:00,2017-05-27 10:20:00+00:00,DME,LED,Arrived,321,2017-05-27 09:31:00+00:00,2017-05-27 10:26:00+00:00
97,70,PG0403,2017-05-27 08:25:00+00:00,2017-05-27 09:20:00+00:00,DME,LED,Arrived,321,2017-05-27 08:30:00+00:00,2017-05-27 09:26:00+00:00
98,71,PG0404,2017-05-27 16:05:00+00:00,2017-05-27 17:00:00+00:00,DME,LED,Arrived,321,2017-05-27 16:07:00+00:00,2017-05-27 17:02:00+00:00


Но можно проще!

In [35]:
try:
    conn = psycopg2.connect(
                            host=config['otus_postgreSQL']['host'],
                            port=config['otus_postgreSQL']['port'],
                            database=config['otus_postgreSQL']['db'],
                            user=config['otus_postgreSQL']['user'],
                            password=str(config['otus_postgreSQL']['pass']))

    query_str = "select * \
        from flights \
        limit 100;"

    df = pd.read_sql_query(query_str, conn)
except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL", error)
finally:
    conn.close()

/var/folders/lk/6f78t4jn60s5_ntqqc2dj0980000gn/T/ipykernel_64615/2899464576.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_str, conn)


In [37]:
df

,flight_id,flight_no,scheduled_departure,scheduled_arrival,departure_airport,arrival_airport,status,aircraft_code,actual_departure,actual_arrival
0,182,PG0402,2017-09-01 09:25:00+00:00,2017-09-01 10:20:00+00:00,DME,LED,Scheduled,321,NaT,NaT
1,1996,PG0335,2017-08-26 06:30:00+00:00,2017-08-26 08:35:00+00:00,DME,JOK,Scheduled,CN1,NaT,NaT
2,5979,PG0384,2017-08-26 09:10:00+00:00,2017-08-26 09:40:00+00:00,DME,BZK,Scheduled,SU9,NaT,NaT
3,8136,PG0138,2017-08-28 07:15:00+00:00,2017-08-28 08:20:00+00:00,VKO,RTW,Scheduled,CR2,NaT,NaT
4,10455,PG0277,2017-09-12 08:45:00+00:00,2017-09-12 12:10:00+00:00,SVO,OVB,Scheduled,773,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...
95,68,PG0405,2017-05-17 06:35:00+00:00,2017-05-17 07:30:00+00:00,DME,LED,Arrived,321,2017-05-17 06:36:00+00:00,2017-05-17 07:32:00+00:00
96,69,PG0402,2017-05-27 09:25:00+00:00,2017-05-27 10:20:00+00:00,DME,LED,Arrived,321,2017-05-27 09:31:00+00:00,2017-05-27 10:26:00+00:00
97,70,PG0403,2017-05-27 08:25:00+00:00,2017-05-27 09:20:00+00:00,DME,LED,Arrived,321,2017-05-27 08:30:00+00:00,2017-05-27 09:26:00+00:00
98,71,PG0404,2017-05-27 16:05:00+00:00,2017-05-27 17:00:00+00:00,DME,LED,Arrived,321,2017-05-27 16:07:00+00:00,2017-05-27 17:02:00+00:00
